In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# --- CONFIGURATION CRITIQUE ---

# Si vous êtes DANS Jupyter (le conteneur Docker), le nom d'hôte est "loki"
# Si vous êtes sur votre PC (en dehors de Docker), le nom d'hôte est "localhost"
LOKI_HOST = "localhost" 
LOKI_PORT = "3100"

# L'erreur venait d'ici : il manquait "http://" au début
BASE_URL = f"http://{LOKI_HOST}:{LOKI_PORT}"

# On construit l'URL complète
LOKI_URL = f"{BASE_URL}/loki/api/v1/query_range"

# On cible le conteneur de la victime (le serveur SSH)
# On cherche les logs contenant "Failed password"
QUERY = '{container="victim_server"} |= "Failed password"'

print(f"Test de connexion vers : {BASE_URL}/ready")
try:
    # Petit test pour voir si Loki répond avant de lancer la grosse requête
    r = requests.get(f"{BASE_URL}/ready")
    if r.status_code == 200:
        print("✅ Connexion à Loki réussie !")
    else:
        print(f"⚠️ Loki répond mais n'est pas prêt (Code: {r.status_code})")
except Exception as e:
    print(f"❌ Impossible de joindre Loki : {e}")

    

Test de connexion vers : http://localhost:3100/ready
✅ Connexion à Loki réussie !


In [2]:
def get_attack_logs():
    # On regarde les 5 dernières minutes
    end_time = datetime.now()
    start_time = end_time - timedelta(minutes=5)
    
    # Paramètres conformes à l'API Loki
    params = {
        'query': QUERY,
        'start': int(start_time.timestamp() * 1e9), # Nanosecondes
        'end': int(end_time.timestamp() * 1e9),
        'limit': 1000,
        'direction': 'BACKWARD'
    }
    
    print(f"🔍 Interrogation de : {LOKI_URL} avec la requête : {QUERY}")
    
    try:
        response = requests.get(LOKI_URL, params=params)
        response.raise_for_status() # Lève une erreur si code != 200
        data = response.json()
        return data['data']['result']
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur API : {e}")
        return []
def analyze_bruteforce(logs):
    if not logs:
        print("📭 Aucun log trouvé.")
        return

    records = []
    for stream in logs:
        for entry in stream['values']:
            # Entry[0] est le timestamp en nanosecondes
            records.append({
                'timestamp': pd.to_datetime(int(entry[0]), unit='ns'), 
                'message': entry[1]
            })
    
    df = pd.DataFrame(records)
    print(f"📋 {len(df)} tentatives de connexion trouvées dans les logs.")

    # On trie par temps
    df = df.sort_values('timestamp')
    df.set_index('timestamp', inplace=True)

    # --- MÉTHODE 1 : Fenêtre glissante (plus précis pour un SIEM) ---
    # On compte combien d'événements surviennent dans une fenêtre de 10 secondes
    df['rolling_count'] = df['message'].rolling('10s').count()
    
    # On définit un seuil bas pour le TP (ex: plus de 3 tentatives en 10s)
    SEUIL = 3
    alerts = df[df['rolling_count'] > SEUIL]
    
    if not alerts.empty:
        print(f"\n🚨 ALERTE SIEM : BRUTEFORCE DÉTECTÉ !")
        print(f"Maximum d'attaques simultanées : {int(df['rolling_count'].max())}")
        print("Dernières alertes :")
        print(alerts[['message', 'rolling_count']].tail(5))
    else:
        print(f"✅ Analyse : RAS. (Max détecté : {int(df['rolling_count'].max())}/10s, Seuil : {SEUIL})")

# Exécution
logs_data = get_attack_logs()
analyze_bruteforce(logs_data)

🔍 Interrogation de : http://localhost:3100/loki/api/v1/query_range avec la requête : {container="victim_server"} |= "Failed password"
📋 336 tentatives de connexion trouvées dans les logs.

🚨 ALERTE SIEM : BRUTEFORCE DÉTECTÉ !
Maximum d'attaques simultanées : 48
Dernières alertes :
                                                                         message  \
timestamp                                                                          
2026-02-17 21:47:58.936098092  Failed password for root from 172.26.0.8 port ...   
2026-02-17 21:47:58.936108592  Failed password for root from 172.26.0.8 port ...   
2026-02-17 21:47:58.936109925  Failed password for root from 172.26.0.8 port ...   
2026-02-17 21:47:58.943194008  Failed password for root from 172.26.0.8 port ...   
2026-02-17 21:47:58.955809717  Failed password for root from 172.26.0.8 port ...   

                               rolling_count  
timestamp                                     
2026-02-17 21:47:58.936098092      

In [3]:
def analyze_bruteforce_v2(logs):
    if not logs:
        print("📭 Loki est vide. Vérifiez 'docker logs victim_server'")
        return

    records = []
    for stream in logs:
        for entry in stream['values']:
            records.append({'ts': pd.to_datetime(int(entry[0]), unit='ns')})
    
    df = pd.DataFrame(records).set_index('ts')
    
    # On regarde les 30 dernières secondes uniquement
    now = datetime.now()
    recent_cutoff = now - timedelta(seconds=30)
    # Note: On convertit en UTC car les logs Docker sont souvent en UTC
    recent_attacks = df[df.index > (recent_cutoff - timedelta(hours=1))] 

    count = len(recent_attacks)
    print(f"🔥 {count} tentatives détectées sur les 30 dernières secondes.")

    if count > 2: # Seuil très bas pour le TP
        print(f"🚨 ALERTE SIEM : BRUTEFORCE EN COURS ({count} tentatives) !")
    else:
        print("✅ Trafic calme.")



In [ ]:
import time
from IPython.display import clear_output

try:
    while True:
        clear_output(wait=True)
        print(f"--- Monitoring SIEM en cours ({datetime.now().strftime('%H:%M:%S')}) ---")
        
        logs_data = get_attack_logs()
        analyze_bruteforce_v2(logs_data)
        
        print("\nProchaine analyse dans 10 secondes... (Arrêtez avec le bouton Stop)")
        time.sleep(1)
except KeyboardInterrupt:
    print("Monitoring arrêté.")

--- Monitoring SIEM en cours (22:48:10) ---
🔍 Interrogation de : http://localhost:3100/loki/api/v1/query_range avec la requête : {container="victim_server"} |= "Failed password"
🔥 48 tentatives détectées sur les 30 dernières secondes.
🚨 ALERTE SIEM : BRUTEFORCE EN COURS (48 tentatives) !

Prochaine analyse dans 10 secondes... (Arrêtez avec le bouton Stop)
